In [ ]:
from ROOT import *
import ctypes
import math

: 

In [2]:
from array import array

In [3]:
#Set resolved fit funtion
gInterpreter.Declare("""
    Double_t resolved_fit(Double_t *xx, Double_t *par)
    {
        Float_t x =xx[0];
        Double_t f = par[8]*TMath::Gaus(x,par[4], par[10]) + TMath::ATan((x-par[9])/par[11])*par[0]*ROOT::Math::crystalball_function(x, par[1], par[2], par[3], par[4])+par[5]*TMath::Erf((x-par[6])/par[7]);
        return f;
    }
""")

gInterpreter.Declare("""
    Double_t merged_fit(Double_t *xx, Double_t *par)
    {
       Double_t temp = 1700;  //quadratic spline
       Double_t x =xx[0], y=(x-temp)/1000.;
       Double_t f;

       if(x<=temp) f = par[0]*TMath::Erf( (x-par[1])/par[2] ) + (par[3] + par[4]*TMath::ATan((x-par[5])/par[6]));
       else {f = merged_fit(&temp, par) + par[7]*y+par[8]*y*y; }
       return f;
    }
""")

True

In [4]:
def GetEffAndError(h_gen,h_reco):
    err_gen = ctypes.c_double(0.0)
    N_gen = h_gen.IntegralAndError(1,400,err_gen)
    #N_gen = h_gen.IntegralAndError(err_gen)
    N_gen = h_gen.Integral()

    err_reco = ctypes.c_double(0.0)
    N_reco = h_reco.IntegralAndError(1,400,err_reco)
    #N_reco = h_reco.IntegralAndError(err_reco)
    #N_reco = h_reco.Integral()

    eff = N_reco/N_gen
    #eff_err = err_reco.value/err_gen.value
    #eff_err = math.sqrt(math.pow(err_reco.value/N_reco,2)+math.pow(err_gen.value/N_gen,2))
    if N_reco==0.0:
        eff_err = 0.0
    else:
        eff_err = math.sqrt(math.pow(err_reco.value/N_reco,2)+math.pow(err_gen.value/N_gen,2))*eff
    return eff, eff_err

    

In [5]:
sample = sam = 'ggh'; year = '2018'
#tags = ['btag','untag','vbftag']
#tags = ['btag']; tag_str = 'btag'
tags = ['untag']; tag_str = 'untag'
#tags = ['vbftag']; tag_str = 'vbftag'
#channels = ['isEE','isMuMu']
channels = ['isEE']; channel_str = 'isEE'
#channels = ['isMuMu']; channel_str = 'isMuMu'
#channels = ['isEE','isMuMu']
#cases = ['merged_tag']; case_str = 'merged_tag'
cases = ['resolved']; case_str = 'resolved'
#cases = ['merged_tag','resolved']
massList = []
for mass in range(200,1000,50):
    massList.append(mass)
for mass in range(1000,1600,100):
    massList.append(mass)
for mass in range(1600,3400,200):
    massList.append(mass)
if(sam=='ggh'):
    pro_str = 'spin0_ggH'
elif(sam=='vbf'):
    pro_str = 'spin0_VBF'
chanName = {'isEE':'eeqq','isMuMu':'mumuqq','2lep':'llqq'}
caseName = {'merged_tag':"Merged",'resolved':"Resolved"}
tagName = {'btag':'b-tagged','untag':'untagged','vbftag':'vbf-tagged'}

In [17]:
#infile = TFile(f'Histos_MakeShape_{sample}_{year}.root')
infile = TFile(f'Histos_MakeShape_{sample}_{year}_1GeVbin.root')
#infile = TFile(f'Histos_MakeShape_{sample}_{year}_Oct10.root')
h_gen = infile.Get("gen_hist")
#h_gen = infile.Get("ggh")

In [18]:
eff_GraphY = {}; eff_GraphYerr = {}
eff_graph = {}
for channel in channels:
    eff_GraphY[channel] = {}; eff_GraphYerr[channel] = {};eff_graph[channel] = {}
    for case in cases:
        eff_GraphY[channel][case] = {}; eff_GraphYerr[channel][case]={}; eff_graph[channel][case]={}
        for tag in tags:
            eff_GraphY[channel][case][tag] = array('f',[]); eff_GraphYerr[channel][case][tag] = array('f',[]); eff_graph[channel][case][tag]=None
            for i, mass in enumerate(massList):
                h_reco = infile.Get(f'reshape_{channel}_{case}_{tag}_{mass}')
                eff, eff_err = GetEffAndError(h_gen,h_reco)
                eff_GraphY[channel][case][tag].append(eff); eff_GraphYerr[channel][case][tag].append(eff_err)
                print(f'eff = {eff} , err = {eff_err} in {channel} {case} {tag} in {sample} with {mass}')


eff = 0.13344687790691215 , err = 0.04921999424885627 in isEE resolved untag in ggh with 200
eff = 0.05489390813047002 , err = 0.00481462858064827 in isEE resolved untag in ggh with 250
eff = 0.17277085196964678 , err = 0.0027547038181697063 in isEE resolved untag in ggh with 300
eff = 0.1946759979511704 , err = 0.001749141460894338 in isEE resolved untag in ggh with 350
eff = 0.09055445470842087 , err = 0.0008503479779145768 in isEE resolved untag in ggh with 400
eff = 0.0020099447372021713 , err = 2.6235216807307762e-05 in isEE resolved untag in ggh with 450
eff = 4.8905264536225296e-06 , err = 5.474679259708804e-07 in isEE resolved untag in ggh with 500
eff = 0.0 , err = 0.0 in isEE resolved untag in ggh with 550
eff = 0.0 , err = 0.0 in isEE resolved untag in ggh with 600
eff = 0.0 , err = 0.0 in isEE resolved untag in ggh with 650
eff = 0.0 , err = 0.0 in isEE resolved untag in ggh with 700
eff = 0.0 , err = 0.0 in isEE resolved untag in ggh with 750
eff = 0.0 , err = 0.0 in isEE 

In [14]:
#ifwrite = False #set this to True if you want to write the eff_graph to a root file
ifwrite = True #set this to True if you want to write the eff_graph to a root file
y_max = 0.1

polyFunctot_resolved = TF1("polyFunctot_resolved",resolved_fit, 400., 3400, 12)
polyFunctot_resolved.SetParameters(.1, -1.5, 2, 200, 300, .04, 1000, 100, .1, 800, 200)

#polyFunctot_resolved_test = TF1("polyFunctot_resolved_test",resolved_fit_crystalbal, 500., 3000, 10)
#polyFunctot_resolved_exp = TF1("polyFunctot_resolved_exp",resolved_fit_exp, 500., 3000, 4)
polyFunctot_merged= TF1("polyFunctot_merged",merged_fit, 400., 3400, 9)
#polyFunctot_merged= TF1("polyFunctot_merged",merged_fit, 200., 3000)
polyFunctot_merged.SetParameters(0.055533, 696.978882, 201.791214, 0.074554, 0.013758, 1489.137451, 272.321777, 0.01, -0.01)
polyFunctot_merged.SetParLimits(0,.003,.2)
polyFunctot_merged.SetParLimits(1,500,1100)
polyFunctot_merged.SetParLimits(2,150,500)
polyFunctot_merged.SetParLimits(3,0,1)
polyFunctot_merged.SetParLimits(4,.003,.2)
polyFunctot_merged.SetParLimits(5,500,1100)
polyFunctot_merged.SetParLimits(6,150,500)
polyFunctot_merged.SetParLimits(7,-.02,.02)
polyFunctot_merged.SetParLimits(8,-.02,.02)
#create TGaph for store and fro Fit 
MultiGraph = {}
eff_graph = {}
eff_graphX = array('f',massList); eff_graphXerr = array('f',[])
for i in eff_graphX:
    eff_graphXerr.append(0)
#colorList = [ROOT.EColor.kGreen,ROOT.EColor.kYellow+2,ROOT.EColor.kRed,ROOT.EColor.kMagenta,ROOT.EColor.kBlue,ROOT.EColor.kCyan]
colorList = [kGreen,kYellow+2,kRed,kMagenta,kBlue,kCyan]
fit_fun_pol5 = TF1('pol5','[0]+[1]*x+[2]*x^2+[3]*x^3+[4]*x^4+[5]*x^5',400,3400)#+[6]*x^6+[7]*x^7',500,3000) #+[8]*x^8+[9]*x^9+[10]*x^10+[11]*x^11',0,3500)
#fit_fun_exp = TF1('exp',"(x/[0] + [1](1- TMath::Exp(-x/[2]))**[3])", 0,5)
#fit_fun_exp = TF1('exp',"([0] + [1]*TMath::Exp(-(x+[2])/[3])", 500,3000)
#fit_fun = TF1('pol11','([0]+[1]*erf((x-[2])/[3]))*([4]+[5]*x+[6]*x*x+[7]*x*x*x)+([8]*gaus(x))',110., 3000)
##fit_fun.SetParameters(0,0,1000,10,0,0,0,0,0,0,0)
#fit_fun.SetParameters(-4.42749e+00,4.61212e+0,-6.21611e+01,1.13168e+02,2.14321e+00,1.04083e-03,4.89570e-07, 0.03, 200, 100)
#fit_fun.SetParLimits(7,0,0.2)
#fit_fun.SetParLimits(8,160,210)
#fit_fun.SetParLimits(9,10,70)
#fit_fun.SetParameters(100,0,0,0,0,0,0,0,0,0,0)
#polyFunctot_merged = TF1('pol5',500., 3000, 9)
if ifwrite:
    if sam=='ggh':
        if case_str !='resolved':
            outfile = TFile(f"./Files/2l2q_Efficiency_spin0_ggH_{year}_merged.root",'recreate'); outfile.cd()
        else:
            outfile = TFile(f"./Files/2l2q_Efficiency_spin0_ggH_{year}_resolved_{channel_str}_{tag_str}.root",'recreate'); outfile.cd()
        #outfile = TFile(f"./2l2q_Efficiency_spin0_ggH_{year}_oneCat.root",'recreate'); outfile.cd()
    elif sam=='vbf':
        if case_str!='resolved':
            outfile = TFile(f"./Files/2l2q_Efficiency_spin0_VBF_{year}_merged.root",'recreate'); outfile.cd()
        else:
            outfile = TFile(f"./Files/2l2q_Efficiency_spin0_VBF_{year}_resolved_{channel_str}_{tag_str}.root",'recreate'); outfile.cd()
lines = array('f',[0,0.01,0.02,.03,0.04])
fit_min = 400; fit_max = 3200
for channel in channels:
    #c = TCanvas('c','c',1300,800)
    #c.SetGrid()
    eff_graph[channel] = {}; MultiGraph[channel] = TMultiGraph()
    leg = TLegend( .64, .80, .97, .99)
    i = 0
    for case in cases:
        eff_graph[channel][case] = {}
        for tag in tags:
            #c = TCanvas('c','c',1300,800)
            #c.SetGrid()
            #c.SetGridlines(lines)
            #print(f'[INFO] this eff in  {channel}_{case}_{tag} = {eff_GraphY[channel][case][tag]}')
            eff_graph[channel][case][tag] = TGraphErrors(len(eff_graphX),eff_graphX,eff_GraphY[channel][case][tag],eff_graphXerr,eff_GraphYerr[channel][case][tag])
            #eff_graph[channel][case][tag] = TGraphErrors(len(eff_graphX),eff_graphX,eff_GraphY,eff_graphXerr,eff_GraphYerr)
            MultiGraph[channel].Add(eff_graph[channel][case][tag],'PL')
            eff_graph[channel][case][tag].SetLineColor(colorList[i]) ; i +=1
            #eff_graph[channel][case][tag].SetLineStyle(0)
            eff_graph[channel][case][tag].SetMarkerSize(1)
            eff_graph[channel][case][tag].SetMarkerStyle(20)
            leg.AddEntry(eff_graph[channel][case][tag],f'{channel}_{case}_{tag}_{sample}',"l")
            #================initilizeParm====================
            polyFunctot_resolved.SetParLimits(0,0,.25)
            polyFunctot_resolved.SetParLimits(1,-2,-1)
            polyFunctot_resolved.SetParLimits(2,1.3,2.5)
            polyFunctot_resolved.SetParLimits(3,50,300)
            polyFunctot_resolved.SetParLimits(4,100,200)
            polyFunctot_resolved.SetParLimits(6,1000,1500)
            polyFunctot_resolved.SetParLimits(7,100,500)
            polyFunctot_resolved.SetParLimits(8,-.05,.05)
            polyFunctot_resolved.SetParLimits(9,100,600)
            polyFunctot_resolved.SetParLimits(10,200,800)
            polyFunctot_resolved.SetParLimits(11,100,1000)
            #=================ResetParm==========================
            if(sam=='vbf' and channel == 'isEE' and case=='resolved' and tag=='btag'):
                if(year=='2016'):
                    print('[INFO] fix parm for vbf_eeqq_resolved_btag in 2016')
                    polyFunctot_resolved.SetParLimits(1,-2,-0.5)
                    polyFunctot_resolved.SetParLimits(2,1.3,3.0)
                    polyFunctot_resolved.SetParLimits(6,1000,1500)
                    polyFunctot_resolved.SetParLimits(7,100,600)
                elif(year=='2017' or year=='2018'):
                    print('[INFO] fix parm for vbf_eeqq_resolved_btag in 2017 and 2018')
                    polyFunctot_resolved.SetParLimits(1,-2.51,-1)
                    polyFunctot_resolved.SetParLimits(2,1.3,3.0)
                    polyFunctot_resolved.SetParLimits(3,50,300)
                    polyFunctot_resolved.SetParLimits(4,50,120)
                    polyFunctot_resolved.SetParLimits(6,900,1300)
                    polyFunctot_resolved.SetParLimits(7,500,1200)
                    polyFunctot_resolved.SetParLimits(9,200,350)
                    polyFunctot_resolved.SetParLimits(10,50,400)
            #elif(sam=='vbf' and channel == 'isEE' and case=='resolved' and tag=='vbftag'):
            #    print('[INFO] fix parm for vbf_eeqq_resolved_vbftag')
            #    polyFunctot_resolved.SetParLimits(1,-2,-0.5)
            #    polyFunctot_resolved.SetParLimits(2,1.0,2.5)
            #    polyFunctot_resolved.SetParLimits(6,350,1200)
            #    polyFunctot_resolved.SetParLimits(7,0,500)
            #    polyFunctot_resolved.SetParLimits(10,50,500)
            elif(sam=='vbf' and channel == 'isMuMu' and case=='resolved' and tag=='btag'):
                if(year=='2018' or year=='2017'):
                    print('[INFO] fix parm for vbf_mumuqq_resolved_btag in 2018')
                    polyFunctot_resolved.SetParLimits(1,-2.51,-1)
                    polyFunctot_resolved.SetParLimits(2,1.3,3.0)
                    polyFunctot_resolved.SetParLimits(3,50,300)
                    polyFunctot_resolved.SetParLimits(4,50,120)
                    polyFunctot_resolved.SetParLimits(6,900,1300)
                    polyFunctot_resolved.SetParLimits(7,500,1200)
                    polyFunctot_resolved.SetParLimits(9,200,350)
                    polyFunctot_resolved.SetParLimits(10,50,400)
                else:
                    print('[INFO] fix parm for vbf_mumuqq_resolved_btag')
                    polyFunctot_resolved.SetParLimits(1,-2,-0.5)
                    polyFunctot_resolved.SetParLimits(2,1.3,3.0)
                    polyFunctot_resolved.SetParLimits(6,1000,1500)
                    polyFunctot_resolved.SetParLimits(7,100,600)
            #elif(sam=='ggh' and channel == 'isMuMu' and case=='resolved' and tag=='vbftag'):
            elif(sam=='ggh' and case=='resolved' and tag=='vbftag'):
                if(year=='2018'):
                    print('[INFO] fix parm for ggh_mumuqq_resolved_vbftag in 2018')
                    polyFunctot_resolved.SetParLimits(1,-2.51,-1)
                    polyFunctot_resolved.SetParLimits(2,1.3,3.0)
                    polyFunctot_resolved.SetParLimits(3,50,100)
                    polyFunctot_resolved.SetParLimits(4,30,90)
                    polyFunctot_resolved.SetParLimits(6,900,1300)
                    polyFunctot_resolved.SetParLimits(7,500,1200)
                    polyFunctot_resolved.SetParLimits(9,220,350)
                    polyFunctot_resolved.SetParLimits(10,150,350)
                else:
                    print('[INFO] fix parm for ggh_mumuqq_resolved_vbftag in 2016 and 2017')
                    polyFunctot_resolved.SetParLimits(1,-2.51,-1)
                    polyFunctot_resolved.SetParLimits(2,1.3,3.0)
                    polyFunctot_resolved.SetParLimits(3,50,100)
                    polyFunctot_resolved.SetParLimits(4,30,90)
                    polyFunctot_resolved.SetParLimits(6,900,1300)
                    polyFunctot_resolved.SetParLimits(7,500,1200)
                    polyFunctot_resolved.SetParLimits(9,220,350)
                    polyFunctot_resolved.SetParLimits(10,150,350)
            elif(sam=='ggh' and case=='resolved' and tag=='btag'):
                print('[INFO] fix parm for ggh_eeqq_resolved_btag')
                polyFunctot_resolved.SetParLimits(1,-2,-0.5)
                polyFunctot_resolved.SetParLimits(2,1.3,3.0)
                polyFunctot_resolved.SetParLimits(6,1000,1500)
                polyFunctot_resolved.SetParLimits(7,100,600)

            #fit
            if(case=='merged_tag'):
                #if(tag=='vbftag' or tag=='btag'):
                if(tag=='vbftag'):
                    eff_graph[channel][case][tag].Fit('pol4',"qwm","",fit_min,fit_max)
                elif(tag=='btag' and year=='2018'):
                    if(channel=='isEE' and sam == 'vbf'):
                        print('[INFO] fix parm for vbf_eeqq_merged_btag')
                        polyFunctot_merged.SetParLimits(0,.005,.2)
                        polyFunctot_merged.SetParLimits(1,500,1100)
                        polyFunctot_merged.SetParLimits(2,150,500)
                        polyFunctot_merged.SetParLimits(3,0,1)
                        polyFunctot_merged.SetParLimits(4,.003,.2)
                        polyFunctot_merged.SetParLimits(5,500,1100)
                        polyFunctot_merged.SetParLimits(6,150,500)
                        polyFunctot_merged.SetParLimits(7,-.048,0)
                        polyFunctot_merged.SetParLimits(8,-.06,0)
                    elif(channel=='isEE' and sam == 'ggh'):
                        print('[INFO] fix parm for ggh_eeqq_merged_btag')
                        polyFunctot_merged.SetParLimits(0,.003,.2)
                        polyFunctot_merged.SetParLimits(1,500,1100)
                        polyFunctot_merged.SetParLimits(2,150,500)
                        polyFunctot_merged.SetParLimits(3,0,1)
                        polyFunctot_merged.SetParLimits(4,.003,.2)
                        polyFunctot_merged.SetParLimits(5,500,1100)
                        polyFunctot_merged.SetParLimits(6,150,500)
                        polyFunctot_merged.SetParLimits(7,-.02,.02)
                        polyFunctot_merged.SetParLimits(8,-.02,.02)
                    else:
                        #polyFunctot_merged.SetParLimits(0,.005,.2)
                        #polyFunctot_merged.SetParLimits(1,500,1100)
                        #polyFunctot_merged.SetParLimits(2,150,500)
                        #polyFunctot_merged.SetParLimits(3,0,1)
                        #polyFunctot_merged.SetParLimits(4,.003,.2)
                        #polyFunctot_merged.SetParLimits(5,400,1000)
                        #polyFunctot_merged.SetParLimits(6,150,500)
                        #polyFunctot_merged.SetParLimits(7,-.0467,0)
                        #polyFunctot_merged.SetParLimits(8,-.05,0)
                        polyFunctot_merged.SetParLimits(0,.003,.2)
                        polyFunctot_merged.SetParLimits(1,500,1100)
                        polyFunctot_merged.SetParLimits(2,150,500)
                        polyFunctot_merged.SetParLimits(3,0,1)
                        polyFunctot_merged.SetParLimits(4,.003,.2)
                        polyFunctot_merged.SetParLimits(5,500,1100)
                        polyFunctot_merged.SetParLimits(6,150,500)
                        polyFunctot_merged.SetParLimits(7,-.02,.02)
                        polyFunctot_merged.SetParLimits(8,-.02,.02)
                    eff_graph[channel][case][tag].Fit(polyFunctot_merged,"qwm","",fit_min, fit_max)
                elif(tag=='btag' and year=='2016'):
                    if(channel=='isEE' and sam == 'vbf'):
                        print('[INFO] fix parm for vbf_eeqq_merged_btag')
                        polyFunctot_merged.SetParLimits(0,.005,.2)
                        polyFunctot_merged.SetParLimits(1,500,1100)
                        polyFunctot_merged.SetParLimits(2,150,500)
                        polyFunctot_merged.SetParLimits(3,0,1)
                        polyFunctot_merged.SetParLimits(4,.003,.2)
                        polyFunctot_merged.SetParLimits(5,500,1100)
                        polyFunctot_merged.SetParLimits(6,150,500)
                        polyFunctot_merged.SetParLimits(7,-.048,0)
                        polyFunctot_merged.SetParLimits(8,-.06,0)
                    elif(channel=='isEE' and sam == 'ggh'):
                        print('[INFO] fix parm for ggh_eeqq_merged_btag')
                        polyFunctot_merged.SetParLimits(0,.003,.2)
                        polyFunctot_merged.SetParLimits(1,500,1100)
                        polyFunctot_merged.SetParLimits(2,150,500)
                        polyFunctot_merged.SetParLimits(3,0,1)
                        polyFunctot_merged.SetParLimits(4,.003,.2)
                        polyFunctot_merged.SetParLimits(5,500,1100)
                        polyFunctot_merged.SetParLimits(6,150,500)
                        polyFunctot_merged.SetParLimits(7,-.015,.02)
                        polyFunctot_merged.SetParLimits(8,-.02,.02)
                    elif(channel=='isMuMu' and sam == 'ggh'):
                        print('[INFO] fix parm for ggh_MuMuqq_merged_btag')
                        polyFunctot_merged.SetParLimits(0,.003,.2)
                        polyFunctot_merged.SetParLimits(1,500,1100)
                        polyFunctot_merged.SetParLimits(2,150,500)
                        polyFunctot_merged.SetParLimits(3,0,1)
                        polyFunctot_merged.SetParLimits(4,.003,.2)
                        polyFunctot_merged.SetParLimits(5,500,1100)
                        polyFunctot_merged.SetParLimits(6,150,500)
                        polyFunctot_merged.SetParLimits(7,-.015,.02)
                        polyFunctot_merged.SetParLimits(8,-.02,.02)
                    else:
                        polyFunctot_merged.SetParLimits(0,.003,.2)
                        polyFunctot_merged.SetParLimits(1,500,1100)
                        polyFunctot_merged.SetParLimits(2,150,500)
                        polyFunctot_merged.SetParLimits(3,0,1)
                        polyFunctot_merged.SetParLimits(4,.003,.2)
                        polyFunctot_merged.SetParLimits(5,500,1100)
                        polyFunctot_merged.SetParLimits(6,150,500)
                        polyFunctot_merged.SetParLimits(7,-.015,.02)
                        polyFunctot_merged.SetParLimits(8,-.02,.02)
                    eff_graph[channel][case][tag].Fit(polyFunctot_merged,"qwm","",fit_min, fit_max)
                elif(tag=='btag' and year=='2017'):
                    print("[INFO] fix parm for btag in 2017")
                    polyFunctot_merged.SetParLimits(0,.005,.2)
                    polyFunctot_merged.SetParLimits(1,500,1100)
                    polyFunctot_merged.SetParLimits(2,150,500)
                    polyFunctot_merged.SetParLimits(3,0,1)
                    polyFunctot_merged.SetParLimits(4,.003,.2)
                    polyFunctot_merged.SetParLimits(5,400,1000)
                    polyFunctot_merged.SetParLimits(6,150,500)
                    polyFunctot_merged.SetParLimits(7,-.0467,0)
                    polyFunctot_merged.SetParLimits(8,-.05,0)
                    if(channel=='isEE'):
                        if(sam == 'ggh'):
                            print("[INFO] fix parm for btag in 2017 isEE ggh")
                            polyFunctot_merged.SetParLimits(5,400,1200)
                            polyFunctot_merged.SetParLimits(7,-.015,0)
                        elif(sam == 'vbf'):
                            print("[INFO] fix parm for btag in 2017 isEE vbf")
                            polyFunctot_merged.SetParLimits(5,400,1200)
                            polyFunctot_merged.SetParLimits(7,-.05,0)
                    eff_graph[channel][case][tag].Fit(polyFunctot_merged,"qwm","",fit_min, fit_max)
                elif(tag=='untag'):
                    polyFunctot_merged.SetParLimits(0,.003,.2)
                    polyFunctot_merged.SetParLimits(1,500,1100)
                    polyFunctot_merged.SetParLimits(2,150,500)
                    polyFunctot_merged.SetParLimits(3,0,1)
                    polyFunctot_merged.SetParLimits(4,.003,.2)
                    polyFunctot_merged.SetParLimits(5,500,1100)
                    polyFunctot_merged.SetParLimits(6,150,500)
                    polyFunctot_merged.SetParLimits(7,-.02,.02)
                    polyFunctot_merged.SetParLimits(8,-.02,.02)
                    eff_graph[channel][case][tag].Fit(polyFunctot_merged,"qwm","",fit_min, fit_max)
                else:
                    eff_graph[channel][case][tag].Fit(polyFunctot_merged,"qwm","",fit_min, fit_max)
            else:
                eff_graph[channel][case][tag].Fit(polyFunctot_resolved,"qwm","",fit_min, fit_max)
                #eff_graph[channel][case][tag].Fit(polyFunctot_resolved_test,"qwm","",500.0, 3000.0)
                #eff_graph[channel][case][tag].Fit(polyFunctot_resolved_exp,"qwm","",500.0, 3000.0)
                #eff_graph[channel][case][tag].Fit('pol6',"qwm","",500.0, 3000.0)
                #eff_graph[channel][case][tag].Fit(fit_fun_pol5,"qwm","",500.0, 3000.0)
            if ifwrite:
                eff_graph[channel][case][tag].Write(f'{pro_str}_{chanName[channel]}_{caseName[case]}_{tagName[tag]}')          


            #c = TCanvas('c','c',1300,800)
            #c.SetGrid()
            #MultiGraph[channel].Draw("A pmc plc")
            #MultiGraph[channel].GetXaxis().SetTitle('M_2l2q [GeV]')
            #MultiGraph[channel].GetYaxis().SetTitle("eff*acc"); MultiGraph[channel].GetYaxis().SetTitleSize(0.025)
            #MultiGraph[channel].SetTitle(f'Channel {channel}')
            #MultiGraph[channel].SetMaximum(y_max)
            #MultiGraph[channel].SetMinimum(0)
            #leg.Draw()
            #c.SetTitle(f'Channel {channel}')
            ##c.SaveAs(f'./plots/{year}/eff_{sam}_{channel}.png')
            #c.SaveAs(f'./plotMay16/{year}/eff_{sam}_{channel}_{case}_{tag}_test.png')
            #c.Close()
    c = TCanvas('c','c',1300,800)
    c.SetGrid()
    #c.SetLogy()
    MultiGraph[channel].Draw("A pmc plc")
    MultiGraph[channel].GetXaxis().SetTitle('M_2l2q [GeV]')
    MultiGraph[channel].GetYaxis().SetTitle("eff*acc"); MultiGraph[channel].GetYaxis().SetTitleSize(0.025)
    MultiGraph[channel].SetTitle(f'Channel {channel}')
    #MultiGraph[channel].SetLogy()
    #MultiGraph[channel].SetMaximum(7e-1)
    #MultiGraph[channel].SetMinimum(1e-0)
    MultiGraph[channel].SetMaximum(1.0)#merged
    #MultiGraph[channel].SetMaximum(0.10)#EE btag resolved
    #MultiGraph[channel].SetMaximum(0.2)#MM untag vbf resolved
    #MultiGraph[channel].SetMaximum(0.25)# untag ggh resolved
    #MultiGraph[channel].SetMaximum(0.05)#MM btag resolved
    #MultiGraph[channel].SetMaximum(0.2)#MM vbftag vbf resolved
    #MultiGraph[channel].SetMaximum(0.01)#MM vbftag ggh resolved
    MultiGraph[channel].SetMinimum(0)
    leg.Draw()
    c.SetTitle(f'Channel {channel}')
    if case!='resolved':
        c.SaveAs(f'./plotMay16/{year}/eff_{sam}_{channel}_merged.png')
    else:
        c.SaveAs(f'./plotMay16/{year}/eff_{sam}_{channel}_{case}_{tag}.png')
    c.Close()

if ifwrite:
    outfile.Write(); outfile.Close()

Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <TCanvas::Print>: png file ./plotMay16/2018/eff_ggh_isEE_resolved_untag.png has been created
